# Hotelling Model + Stock Pollution Model
**Exercise Session Resource Economics (Spring Term 2025)** \
Raul Hochuli (raul.hochuli@unibas.ch)\ 

Please remember to **send me your solutions (or what you have so far) by email**:
- Hotelling by Sunday 16th March
- Stock Pollution by Sunday 23rd March


In [32]:
# Packages used in the notebook
import numpy as np
import pandas as pd
import scipy.optimize as opt
import plotly.graph_objects as go
from plotly.subplots import make_subplots

## 1 Hotelling Model

An oil drilling company extracts $q$ amount of units in year $t$ of the initial oil repository $S_0$. There is no new discovery or exploration taking place, hence $S_{t+1} = S_t - q_t$ for all ($\forall$) $t$. A unit of oil is priced in the market at $p_t$ and costs a constant marginal cost of $c$ to extract. The discount factor $\rho^t = 1/(1+r)^t$ is given with a constant interest rate over time. In a competitive market, the drilling company would face an inverse demand function of $p_t = a-bq_t$. We consider a finite but unknown extraction window $t = 0,1,... T$ and assume that any reserves at $T$ will be worthless. 


**1a)**\
First, assume that the oil drilling company is operating as a monopolist. Make a list with all model components for the optimization problem, classify them (e.g. parameters, variables etc.) and define their range. 

- What implications does the assumption of a monopolist agent have on the optimization problem? 
- What is the interpretation you can derive from the parameter $a$?
- Are the assumptions for a monopolist market and reserves having no value after $T$ realistic (no right or wrong answer - just think about the current structure of the oil market(s))? 

*Solution*
| **Description**       | **Variable**                  | **Range**                  |
|----------------------|----------------------------|---------------------------|
| Time Indicator      | $t$                         | [0, ..., T]          |
| Parameter          | $r, a, b, c, (\rho)$               | constant, (r > -1 ) |
| Decision variable  | $q_t$                   | $ 0 < q_t \leq S_0 $ |
| State variable     | $S_t, p_t$              | $ 0 \leq S_t, p_t $    |
| -                  | -                           | -                          |
| **Objective function** | $\sum_t^T \rho^t(p_t(q_t) - c)q_t $ | -|
| **Constraints**     | $\sum_t^T q_t = S_0 $    | -                          |


\
***Monopolist Assumption:*** When assuming a monopolist market, the price is not given by the market but determined by the quantity the monopolist is selling, hence it is defined as $p(q_t)$ and dependent on $q_t$.

\
***Chock-off price / backstop technology***: The parameter gives the marginal price for the last unit of $q$ sold on the market. This means for any price higher than $a$, no more oil could be sold and the consumers will switch to another technology / solution as a substitute.

**1b)**\
Now, assume $S_0 = 100, a=10, b=1, c = 1, r = 0.05$. Build an optimization model which models the monopolistic price schedule and calculates the present value of the return of extraction for the monopolist. 

- How long does it take for the resource to be fully depleted? (Hint: Start by defining $T$ arbitrarily large so enough time can pass for $q_t$ to actually reach zero)
- What is the initial price $p_0$ and the objective value of your model?
- Visualize the values for $q_t, p_t$ and $S_t$ over time and compare the plots to your lecture slides.

In [33]:
def hotelling_mono(T = 50, S0 = 100, a = 10, b = 1, c = 1, r = 0.05):
    # Sets & Parameters --------------------------------
    # T = 50
    # S0 = 100
    # a = 10
    # b = 1
    # c = 1
    # r = 0.05

    rho = 1/(1+r)**np.arange(T+1)

    # State Variables -----------------------------------

    def f_S(i_q):
        S = np.zeros(T+1)
        S[0] = S0
        for t in range(1, T+1):
            S[t] = S[t-1] - i_q[t-1]
        return S

    def f_p(i_q):
        return a - b*i_q


    # Objective Function -------------------------------
    def f_obj(i_q):
        obj_value = sum(rho[t] * (i_q[t] * (f_p(i_q)[t] - c)) for t in range(T+1))
        return -1* obj_value


    # Decision Variables -------------------------------
    q_start = np.full(T+1, 1)
    bnds = [(0, S0) for t in range(T+1)]

    # Constraints --------------------------------------
    def c_total_extr(i_q):
        return S0 - sum([i_q[t] for t in range(T+1)])
    cnstr = [
        {'type': 'eq', 'fun': c_total_extr},
    ]

    # Run Optimization ---------------------------------
    results = opt.minimize(f_obj, q_start, bounds=bnds, constraints=cnstr,
                        options={'disp': False}
                        )

    # Results ------------------------------------------
    q_opt = results.x
    S_opt = f_S(q_opt)
    p_opt = f_p(q_opt)
    npv_opt = f_obj(q_opt) # also possible to calculcate this array by hand: [rho[t] * (q_opt[t] * (a-b*q_opt[t]) - c*q_opt[t]) for t in range(T+1)]

    df = pd.DataFrame({
        't': range(T+1),
        'q_opt': q_opt,
        'S_opt': S_opt,
        'p_opt': p_opt,
        'npv_opt': npv_opt
    })

    return df

df_hotelling_mono = hotelling_mono()
print(f'objective value: \t{df_hotelling_mono["npv_opt"].sum()}; \np_opt[0]: \t\t{df_hotelling_mono["p_opt"][0]}')


# Plot Results -------------------------------------
fig = make_subplots(rows=1, cols=3, subplot_titles=('q_opt', 'p_opt', 'S_opt'))
def add_trace_to_subplot_hotelling(fig, df, legend_name):
    fig.add_trace(go.Scatter(x=df['t'], y=df['q_opt'], mode='lines', name=f'q_{legend_name}', legendgroup=legend_name, showlegend=True), row=1, col=1)
    fig.add_trace(go.Scatter(x=df['t'], y=df['p_opt'], mode='lines', name=f'p_{legend_name}', legendgroup=legend_name, showlegend=True), row=1, col=2)
    fig.add_trace(go.Scatter(x=df['t'], y=df['S_opt'], mode='lines', name=f'S_{legend_name}', legendgroup=legend_name, showlegend=True), row=1, col=3)
    
    return fig

add_trace_to_subplot_hotelling(fig, df_hotelling_mono, 'base')

fig.update_layout(title='Hotelling_mono', xaxis_title='t', yaxis_title='value', template='plotly_white')
fig.write_html('hotelling_mono.html')  
fig.show()


objective value: 	-15905.106926957316; 
p_opt[0]: 		6.162161474654469


**1c)** \
Now, the oil drilling company is in a *competitive market*. 
- Without going into math or code, what is the main difference between an optimization model for a monopolist vs. a competitive market? 
- To find a numerical model that we can optimize, we first need to derive the Hotelling-Rule with pen and paper and then transfer it to optimization software. (Hint: You can use the Lagrangian $L$ to set up the optimization problem. Calculate the first order derivatives for $\frac{\partial L}{\partial q_0}$ and $\frac{\partial L}{\partial q_t}$ to derive the Hotelling-rule that defines $p_t$)
- What assumption has to be given for the Hotelling-rule?

$$\max_{q_t} \sum_{t=0}^T \rho^t (p_t q_t - cq_t) \qquad {s.t.}\qquad\sum_{t=0}^Tq_t = S_0
\\[3em]
L = \sum_{t=0}^T \rho^t (p_t q_t - cq_t) - \lambda \left( \sum_{t=0}^Tq_t - S_0 \right)

*Solution*
$$
FOC: \\
\frac{\partial L}{\partial q_0} = \underbrace{\rho^0}_{\frac{1}{(1+r)^0} = 1} (p_0 - c) - \lambda = 0 \qquad \rightarrow  \lambda = p_0 - c 
\\
\frac{\partial L}{\partial q_t} = \rho^t (p_t - c) - \lambda = 0 \qquad \rightarrow p_t = c+ \underbrace{\lambda}_{p_0-c}(1+r)^t
\\
p_t = \underbrace{c}_{I} + \underbrace{(p_0 -c)(1+r)^t}_{II}\\
$$
***Main differences***: In a monopolist case, your decision variable is $q$, an entire array of extractions over $t$ which determined $p$. In a competitive market scenario, how much you as one of many market participants extracts depends on the price $p_t$. Because the price follows a predefined pathway, you no longer optimze using an array, but a single initial price value to solve your model.

***Key assumption***: We have to assume that the stock does not increase (discovery), is fully extracted (no valuable reserves left) and that extraction costs remain constant.

**1d)** \
Build a new model to compute the price and extraction schedule under the Hotelling-Rule in the competitive market (Hint: Now take $p_0$ as the only "decision variable" and define the other components endogenously). 

Visualize the results of the otherwise identical monopolist case to spot the differences. Given the model results would you prefer a market with a monopolist or perfect competition?

In [34]:
# Modelling Hotelling in perfect market competition can be tricky! This is why I want to show you first a version of the model, 
# which runs through but gives faulty results. 

def hotelling_compmarket(T = 50, S0 = 100, a = 10, b = 1, c = 1, r = 0.05):

    # Sets & Parameters --------------------------------
    # rho = 1/(1+r)**np.arange(T+1)
    rho = np.array([1/(1+r)**t for t in range(T+1)])

    # State Variables -----------------------------------
    def f_p(i_p0):
        p = np.zeros(T+1)
        p[0] = i_p0
        for t in range(1, T+1):
            p[t] = c + (i_p0 - c) * (1 + r) ** t
        return p
    
    def f_q(i_p0):
        p = f_p(i_p0)
        return (a - p ) / b
    
    
    # Objective Function -------------------------------
    def f_obj(i_p0):
        q = f_q(i_p0)
        p = f_p(i_p0)

        obj_value = sum(rho[t] * (q[t] * (p[t] - c)) for t in range(T+1))
        # obj_value = sum(rho * (q * (p - c)))
        return -1* obj_value

    # Decision Variables -------------------------------
    p0_start = a/2
    bnds = [(0, a)]

    # Constraints --------------------------------------
    def c_total_extr(i_p0):
        q = f_q(i_p0)
        return S0 - sum(q)
   
    cnstr = [
        {'type': 'eq', 'fun': c_total_extr, }, 

    ]

    # Run Optimization ---------------------------------
    results = opt.minimize(f_obj, p0_start, bounds=bnds, 
                           constraints=cnstr,
                           options={'disp': False}
                        )
    
    # Results ------------------------------------------
    p0_opt = results.x[0]
    q_opt = f_q(p0_opt)
    p_opt = f_p(p0_opt)

    def f_S(i_q): 
        S = np.zeros(T+1)
        S[0] = S0
        for t in range(1, T+1):
            S[t] = S[t-1] - i_q[t-1]
        return S
    S_opt = f_S(q_opt)

    df = pd.DataFrame({
        't': range(T+1),
        'q_opt': q_opt,
        'S_opt': S_opt,
        'p_opt': p_opt,
    })

    return df


df_hotelling_compmarket = hotelling_compmarket()


# Plot Results -------------------------------------
fig_mono_compmarket = make_subplots(rows=1, cols=3, subplot_titles=('q_opt', 'p_opt', 'S_opt'))

add_trace_to_subplot_hotelling(fig_mono_compmarket, df_hotelling_mono, 'mono')
add_trace_to_subplot_hotelling(fig_mono_compmarket, df_hotelling_compmarket, 'compmarket')

fig_mono_compmarket.update_layout(title='Hotelling_mono vs. Hotelling_compmarket', xaxis_title='t', yaxis_title='value', template='plotly_white')
fig_mono_compmarket.write_html('hotelling_compmarket_wrongresults.html')
fig_mono_compmarket.show()

# print(df_hotelling_compmarket)


C:\Users\hocrau00\AppData\Local\Temp\ipykernel_18816\1287254770.py:13: DeprecationWarning:

Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)

C:\Users\hocrau00\AppData\Local\Temp\ipykernel_18816\1287254770.py:15: DeprecationWarning:

Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)



*Explanaiton for solution*: \
Modelling Hotelling in perfect market competition can be tricky! This is why I want to show you first a version of the model, which runs through but gives faulty results. For interested students, I listed all the debugging steps I took to get reasonable results. **You can jump directly to step 4, if you are only interested the final fix.**
1. Runing the model with just the constraint *c_total_extr()* gives negative values for q and S. We choose to add additional constraints, which however do not solve the problem. 

2. Depending on your package versioning that you use, you might receive the following warning: \
        ```DeprecationWarning:
        Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
        ``` 
        Through some debugging you can find out, that $p_{opt}$ gets passed into the functions f_p() etc. as a 1D array, which causes a warning because it is used as single value object within the functions. You can solve this by calling the first and only value of the array using i_p0[0]. 

3. Optimization problems in software can have wildly varying results dependent on the solver "engine" and starting values. scipy.optimize offers a large number of solvers. If your results look strange, it is a good idea to check different methods. 

4. Despite all these debugging efforts and additional constraints, $q$ and $S$ still have a negative range. Since $S$ is dependent on $q$ we focus on $f_q()$ to solve our issue. A possibility is to simply set the constraint of non negativity into the function of the state variable itself. 

You don't need to learn all these steps by heart! They are more an example on how you can go about debugging your models. 
       

In [35]:
def hotelling_compmarket(T = 50, S0 = 100, a = 10, b = 1, c = 1, r = 0.05):

    # Sets & Parameters --------------------------------
    # rho = 1/(1+r)**np.arange(T+1)
    rho = np.array([1/(1+r)**t for t in range(T+1)])

    # State Variables -----------------------------------
    def f_p(i_p0):
        p = np.zeros(T+1)
        p[0] = i_p0
        for t in range(1, T+1):
            p[t] = c + (i_p0 - c) * (1 + r) ** t
        return p
    
    def f_q(i_p0):
        p = f_p(i_p0)
        q = (a - p) / b
        q[q<0] = 0                  # 4) This line subselects all negative q values given the price development and forces them to be 0. 
        return q
    
    
    # Objective Function -------------------------------
    def f_obj(i_p0):
        i_p0_val = i_p0[0]      # 2) call i_p0 truely as value not as 1D array
        q = f_q(i_p0_val)
        p = f_p(i_p0_val)
        # q, p = f_q(i_p0), f_p(i_p0)

        obj_value = sum(rho[t] * (q[t] * (p[t] - c)) for t in range(T+1))
        # obj_value = sum(rho * (q * (p - c)))
        return -1* obj_value

    # Decision Variables -------------------------------
    p0_start = a/2
    bnds = [(0, a)]

    # Constraints --------------------------------------
    def c_total_extr(i_p0):
        i_p0_val = i_p0[0]
        q = f_q(i_p0_val)
        return S0 - sum(q)
    
    def c_non_negative_S(i_p0):           # 1) Additional Constraint 1
        i_p0_val = i_p0[0]
        q = f_q(i_p0_val)
        S = np.zeros(T+1)
        S[0] = S0
        for t in range(1, T+1):
            S[t] = S[t-1] - q[t-1]
        return S
    
    def c_non_negative_q(i_p0):     # 1) Additional Constraint 2, which does not really solve the problem 
        i_p0_val = i_p0[0]
        q = f_q(i_p0_val)
        return q
    
    cnstr = [
        {'type': 'eq', 'fun': c_total_extr, }, 
        {'type': 'ineq', 'fun': c_non_negative_S,},
        {'type': 'ineq', 'fun': c_non_negative_q}, 
    ]

    # Run Optimization ---------------------------------
    results = opt.minimize(f_obj, p0_start, bounds=bnds, 
                           constraints=cnstr,
                        #    method='COBYQA',           # 3) maybe you need a different solver method / engine to get solid results
                           options={'disp': False}
                        )
    
    # Results ------------------------------------------
    p0_opt = results.x[0]
    q_opt = f_q(p0_opt)
    p_opt = f_p(p0_opt)

    def f_S(i_q): 
        S = np.zeros(T+1)
        S[0] = S0
        for t in range(1, T+1):
            S[t] = S[t-1] - i_q[t-1]
        return S
    S_opt = f_S(q_opt)

    df = pd.DataFrame({
        't': range(T+1),
        'q_opt': q_opt,
        'S_opt': S_opt,
        'p_opt': p_opt,
    })

    return df


df_hotelling_compmarket = hotelling_compmarket()
df_hotelling_mono = hotelling_mono()


# Plot Results -------------------------------------
fig_mono_compmarket = make_subplots(rows=1, cols=3, subplot_titles=('q_opt', 'p_opt', 'S_opt'))
add_trace_to_subplot_hotelling(fig_mono_compmarket, df_hotelling_mono, 'mono')
add_trace_to_subplot_hotelling(fig_mono_compmarket, df_hotelling_compmarket, 'compmarket')

fig_mono_compmarket.update_layout(title='Hotelling_mono vs. Hotelling_compmarket', xaxis_title='t', yaxis_title='value', template='plotly_white')
fig_mono_compmarket.write_html('hotelling_compmarket_corrected.html')
fig_mono_compmarket.show()




## 2 Stock Pollution

Let's now consider stock pollution problems. 

A rural municipality has an agricultural sector using ample amounts of fertilizer to increase crop yield. A unit of fertilizer emissions $e$ benefits the municipality (e.g. crops produced and sold) through the benefit fuction $U(\cdot)$ but also causes polluting residue which flows into the nearby lake increasing the pollution stock $S_t$. Each year the stock causes damages $D(\cdot)$ and is reduced by a certain amount by bacteria $f(S_t)$, depending on the total pollution stock. The total social welfare $W$ is determined by $U$ and $D$ and discounted using the constant discount rate $r$ and the remaining (constant) model parameters.
$$
    W = \sum_{t=0}^{\infty} \rho^t( U(e_t) - D(S_t)) 
    \\[2em]
    S_{t+1} = S_t - f(S_t) + e_t 
    \\[1em]
    f(S_t) = aS_t
    \\[1em]
    U(e_t) = e^b_t
    \\[1em]
    D(S_t) = cS_t^d
    \\[1em]
    \rho_t = \frac{1}{(1+r)^t}
$$



**2a)**\
First build an table with all the components of the optimization problem (e.g. parameters, variables etc.) including the optimization function and constraints

*Solution* 
| **Description**       | **Variable**                  | **Range**                  |
|----------------------|----------------------------|---------------------------|
| Time Indicator      | $t$                         | [0, ..., T]          |
| Parameter          | $r, a, b, c, d, (\rho)$               | constant, (r > -1 ) |
| Decision variable  | $e_t$                   | $ 0 < q_t \leq S_0 $ |
| State variable     | $S_t,$              | $ 0 \leq S_t, p_t $    |
| -                  | -                           | -                          |
| **Objective function** |     $\max\limits_{{e_t}} W = \sum_{t=0}^{\infty} \rho^t( U(e_t) - D(S_t))$ | -|
| **Constraints**     | $S_{t+1} = S_t - \alpha S_t + e_t$    | -                          |


**2b)**\
Build an optimization model to find the social optimal emission rate over time. Use the parameters below and use a model time range of $t = 0, ..., T$.
$$
S_0 = 0; \qquad a = 0.1; \qquad b = 0.6; \qquad c = 0.00008, \qquad d = 2; \qquad r = 0.015; \qquad T = 50
$$

Plot $e$ and $S$ over time. Is there something striking in the visualisation?


In [36]:

def stockpollution(T = 50, S0 = 0, a = 0.1, b = 0.6, c = 0.00008, d = 2, r = 0.015 ):
    # Sets & Parameters --------------------------------
    # Define a discount factor
    rho = 1 / (1 + r) ** np.arange(T + 1)

    # State Variables -----------------------------------
    def f_S(i_e):
        S = np.zeros(T + 1)
        S[0] = S0
        for t in range(1, T + 1):
            S[t] = S[t-1] - a * S[t-1] + i_e[t-1]
        return S

    def f_W(i_e):
        S = f_S(i_e)
        W = np.zeros(T + 1)
        for t in range(T + 1):
            W[t] = rho[t] * ( (i_e[t] ** b) - (c * (S[t] ** d)) )
        return W

    # Objective Function -------------------------------
    def f_obj(i_e):
        W = f_W(i_e)
        return -1 * np.sum(W)


    # Constraints --------------------------------------
    cnstr = []

    # Decision Variables -------------------------------
    e_start = np.full(T+1, S0)
    bnds = [(0, 200) for t in range(T+1)]

    # Run Optimization ---------------------------------
    results = opt.minimize(f_obj, e_start, bounds=bnds, constraints=cnstr, 
                           options={'disp': False})

    # Get the optimal results
    e_opt = results.x
    S_opt = f_S(e_opt)
    W_opt = f_W(e_opt)

    # Create results dataframe
    df = pd.DataFrame({
        't': np.arange(T + 1),
        'e': e_opt,
        'S': S_opt,
        'W': W_opt
    })
    return df

df  = stockpollution()


# Plot Results -------------------------------------
# fig = make_subplots(rows=1, cols=3, subplot_titles=('q_opt', 'p_opt', 'S_opt'))
fig = go.Figure()
fig.add_trace(go.Scatter(x=df['t'], y=df['e'], mode='lines', name='e_opt', showlegend=True))
fig.add_trace(go.Scatter(x=df['t'], y=df['S'], mode='lines', name='S_opt', showlegend=True))
fig.add_trace(go.Scatter(x=df['t'], y=df['W'], mode='lines', name='W_opt', showlegend=True))
fig.update_layout(title='Stock Pollution', xaxis_title='time' , yaxis_title='value', template='plotly_white')

fig.write_html('stockpollution.html')
fig.show()


## 3 For More Advanced Students

**3a) Hotelling, parameter effects**\
If not done already, structure your monopolist hotelling model from before as a function (with the parameters as input) that returns the values for $q_t, p_t$, $S_t$ and the objective value over time $t$ as a dataframe. Rerun the model with different specifications to see how different parameter changes alter the optimal extraction schedule an price path of the resource.

- Increase interest rate: $r^* = 0.07$
- Increase resource stock: $S_0^* = 150$
- Change in the inverse demand function with $b^* = 0.5$
- Increase in extraction cost with $c^* = 2$


In [ ]:
df_hotmono_base = hotelling_mono()
df_hotmono_incrint = hotelling_mono(r=0.07)
df_hotmono_incrstock = hotelling_mono(S0=150)
df_hotmono_decrbparamm = hotelling_mono(b=0.5)
df_hotmono_incrcost = hotelling_mono(c=2)

# Plot Results -------------------------------------
fig_compare = make_subplots(rows=1, cols=3, subplot_titles=('q_opt', 'p_opt', 'S_opt'))
def add_trace_to_subplot_hotelling(fig, df, legend_name):
    fig.add_trace(go.Scatter(x=df['t'], y=df['q_opt'], mode='lines', name=f'q_{legend_name}', legendgroup=legend_name, showlegend=True), row=1, col=1)
    fig.add_trace(go.Scatter(x=df['t'], y=df['p_opt'], mode='lines', name=f'p_{legend_name}', legendgroup=legend_name, showlegend=True), row=1, col=2)
    fig.add_trace(go.Scatter(x=df['t'], y=df['S_opt'], mode='lines', name=f'S_{legend_name}', legendgroup=legend_name, showlegend=True), row=1, col=3)
    
    return fig

fig_compare = add_trace_to_subplot_hotelling(fig_compare, df_hotmono_base, 'base')
fig_compare = add_trace_to_subplot_hotelling(fig_compare, df_hotmono_incrint, 'incr_rparam')
fig_compare = add_trace_to_subplot_hotelling(fig_compare, df_hotmono_incrstock, 'incr_stock')
fig_compare = add_trace_to_subplot_hotelling(fig_compare, df_hotmono_decrbparamm, 'decr_bparam')
fig_compare = add_trace_to_subplot_hotelling(fig_compare, df_hotmono_incrcost, 'incr_cost')

fig_compare.update_layout(title='Hotelling_mono, compare parameter changes', xaxis_title='t', yaxis_title='value', template='plotly_white')
fig_compare.write_html('hotelling_mono_compare.html')
fig_compare.show()


**3b) Hotelling, including ban of extraciton** \
Consider again the case of the monopolist. We now want to take a first step into modeling policy making. Assume the local government imposes legislation that bans all extraction after $T^* = 30$ years. What effect would this have on the extraction schedule given the Hotelling-rule?

In [38]:
def hotelling_mono(T=50, S0=100, a=10, b=1, c=1, r=0.05, extr_ban=None):
    # Sets & Parameters --------------------------------
    rho = 1 / (1 + r) ** np.arange(T + 1)
    
    # State Variables -----------------------------------
    def f_S(i_q):
        S = np.zeros(T + 1)
        S[0] = S0
        for t in range(1, T + 1):
            S[t] = S[t - 1] - i_q[t - 1]
        return S

    def f_p(i_q):
        return a - b * i_q

    # Objective Function -------------------------------
    def f_obj(i_q):
        obj_value = 0
        for t in range(T + 1):
            if extr_ban is not None and t >= extr_ban: 
                i_q[t] = 0
            obj_value += rho[t] * (i_q[t] * (f_p(i_q)[t] - c))
        return -1 * obj_value

    # Decision Variables -------------------------------
    q_start = np.full(T + 1, 1)
    bnds = [(0, S0) for t in range(T + 1)]

    # Constraints --------------------------------------
    def c_total_extr(i_q):
        return S0 - np.sum(i_q)

    cnstr = [{'type': 'eq', 'fun': c_total_extr}]
    
    # Run Optimization ---------------------------------
    results = opt.minimize(f_obj, q_start, bounds=bnds, constraints=cnstr, 
                        #    method='COBYQA',  
                           options={'disp': False})

    # Results ------------------------------------------
    q_opt = results.x
    S_opt = f_S(q_opt)
    p_opt = f_p(q_opt)
    npv_opt = [rho[t] * (q_opt[t] * (a - b * q_opt[t]) - c * q_opt[t]) for t in range(T + 1)]

    df = pd.DataFrame({
        't': range(T + 1),
        'q_opt': q_opt,
        'S_opt': S_opt,
        'p_opt': p_opt,
        'npv_opt': npv_opt
    })

    return df


# Plot Results -------------------------------------
df_hotelling_mono = hotelling_mono()
df_hotelling_mono_ban = hotelling_mono(extr_ban=30)

fig = make_subplots(rows=1, cols=3, subplot_titles=('q_opt', 'p_opt', 'S_opt'))
add_trace_to_subplot_hotelling(fig, df_hotelling_mono, 'mono_base')
add_trace_to_subplot_hotelling(fig, df_hotelling_mono_ban, 'mono_extrban')

fig.update_layout(title = 'Hotelling model base vs ban at T*', xaxis_title='t', yaxis_title='value', template='plotly_white')
fig.write_html('hotelling_mono_extrban.html')
fig.show()


*Explanaiton for solution*: \
Because the monopolist knows, she cannot fully extract the resource over an optimal time, she increases the extraction rate (causing lower prices) to get as much profit during the shortend time frame as possible. This undesired effect is also know as the "green paradox" or the "rush to burn"

**3c) Hotelling, writing in modern python style** \
This is now for the students who are *deeply interested in coding practices* (way beyond our course objective) and have extensive prior knowledge! In recent years, it became fashionable to write your python code using classes (with attributes and methods). Try to refactor your model code for the monopolist hotelling model using python classes. 

In [39]:

class HotellingModel_Mono:
    def __init__(self, T=50, S0=100, a=10, b=1, c=1, r=0.05):
        # Initialize model parameters
        self.T = T
        self.S0 = S0
        self.a = a
        self.b = b
        self.c = c
        self.r = r
        self.rho = 1 / (1 + r) ** np.arange(T + 1)
        self.results = None
        self.df = None

    def f_S(self, i_q):
        S = np.zeros(self.T + 1)
        S[0] = self.S0
        for t in range(1, self.T + 1):
            S[t] = S[t - 1] - i_q[t - 1]
        return S

    def f_p(self, i_q):
        return self.a - self.b * i_q

    def f_obj(self, i_q):
        obj_value = sum(self.rho[t] * (i_q[t] * (self.f_p(i_q)[t] - self.c)) for t in range(self.T + 1))
        return -obj_value

    def c_total_extr(self, i_q):
        return self.S0 - sum([i_q[t] for t in range(self.T + 1)])

    def optimize(self):
        q_start = np.full(self.T + 1, 1)
        bnds = [(0, self.S0) for _ in range(self.T + 1)]
        cnstr = [{'type': 'eq', 'fun': self.c_total_extr}]

        results = opt.minimize(self.f_obj, q_start, bounds=bnds, constraints=cnstr, options={'disp': False})

        self.results = results
        q_opt = results.x
        S_opt = self.f_S(q_opt)
        p_opt = self.f_p(q_opt)
        npv_opt = self.f_obj(q_opt)

        self.df = pd.DataFrame({
            't': range(self.T + 1),
            'q_opt': q_opt,
            'S_opt': S_opt,
            'p_opt': p_opt,
            'npv_opt': npv_opt
        })

    def plot_results(self, filename='hotelling_mono.html'):
        if self.df is None:
            raise ValueError("Run the optimize method first to generate results.")
        
        fig = make_subplots(rows=1, cols=3, subplot_titles=('q_opt', 'p_opt', 'S_opt'))
        fig.add_trace(go.Scatter(x=self.df['t'], y=self.df['q_opt'], mode='lines', name='q_opt', showlegend=False), row=1, col=1)
        fig.add_trace(go.Scatter(x=self.df['t'], y=self.df['p_opt'], mode='lines', name='p_opt', showlegend=False), row=1, col=2)
        fig.add_trace(go.Scatter(x=self.df['t'], y=self.df['S_opt'], mode='lines', name='S_opt', showlegend=False), row=1, col=3)

        fig.update_layout(title='Hotelling Model', xaxis_title='t', yaxis_title='Value', template='plotly_white')
        fig.write_html(filename)
        fig.show()

    def compare_parameters(self, other_models, filename='hotelling_mono_compare.html'):
        """Compare results of different model variations"""
        fig = make_subplots(rows=1, cols=3, subplot_titles=('q_opt', 'p_opt', 'S_opt'))

        for model, label in other_models:
            model.plot_subtrace(fig, label)

        fig.update_layout(title='Hotelling Model - Parameter Comparison', xaxis_title='t', yaxis_title='Value', template='plotly_white')
        fig.write_html(filename)
        fig.show()

    def plot_subtrace(self, fig, label):
        """Helper function to add model results to a subplot"""
        fig.add_trace(go.Scatter(x=self.df['t'], y=self.df['q_opt'], mode='lines', name=f'q_{label}', legendgroup=label, showlegend=True), row=1, col=1)
        fig.add_trace(go.Scatter(x=self.df['t'], y=self.df['p_opt'], mode='lines', name=f'p_{label}', legendgroup=label, showlegend=True), row=1, col=2)
        fig.add_trace(go.Scatter(x=self.df['t'], y=self.df['S_opt'], mode='lines', name=f'S_{label}', legendgroup=label, showlegend=True), row=1, col=3)

# Example usage
if __name__ == "__main__":
    # Base model
    model_base = HotellingModel_Mono()
    model_base.optimize()
    # model_base.plot_results('hotelling_mono.html')

    # Variations
    model_incr_rparam = HotellingModel_Mono(r=0.07)
    model_incr_rparam.optimize()

    model_incr_stock = HotellingModel_Mono(S0=150)
    model_incr_stock.optimize()

    model_decr_bparamm = HotellingModel_Mono(b=0.5)
    model_decr_bparamm.optimize()

    model_incr_cost = HotellingModel_Mono(c=2)
    model_incr_cost.optimize()

    # Compare models
    model_base.compare_parameters(
        [(model_incr_rparam, 'incr_rparam'), (model_incr_stock, 'incr_stock'), (model_decr_bparamm, 'decr_bparam'), (model_incr_cost, 'incr_cost')],
        filename='hotelling_mono_compare.html'
    )
